## Семинар 13: "Обучение с подкреплением 1"

ФИО: Вербин Олег

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [1]:
!pip install gym

In [2]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [3]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [5]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 0
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [7]:
env.reset()

0

In [8]:

env.step(action_to_i['right'])
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [9]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [10]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [11]:
# Используя фиксированную value_function и жадную стратегию получите policy

def extract_policy(v, gamma = 1.0):
    ext_v=np.empty((v.shape[0]+2,v.shape[1]+2))
    ext_v[1:-1,1:-1]=v
    ext_v[0,1:-1]=v[0]
    ext_v[-1,1:-1]=v[-1]
    ext_v[1:-1,0]=v[:,0]
    ext_v[1:-1,-1]=v[:,-1]
    
    tmp=np.empty((v.shape[0],v.shape[1],4))
    tmp[:,:,0]=ext_v[1:-1,:-2]
    tmp[:,:,1]=ext_v[2:,1:-1]
    tmp[:,:,2]=ext_v[1:-1,2:]
    tmp[:,:,3]=ext_v[:-2,1:-1]
    
    policy=np.argmax(tmp,axis=-1)
    return policy.reshape(-1)

In [12]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10
def compute_policy_v(env, policy, old_v, gamma=1.0):
    eps=10
    old_v=old_v.reshape(-1)
    while eps>1e-10:
        eps=0
        for ind,v in enumerate(old_v):
            new_v=0
            for p,vs,_,_ in env.P[ind][policy[ind]]:
                new_v+=p*old_v[vs]
            eps=max(abs(v-new_v),eps)
            old_v[ind]=new_v
    return old_v.reshape(4,4)

In [13]:
def policy_iteration(env, gamma = 1.0):
    max_iterations = 100000
    gamma = 1.0
    policy=np.zeros(16)
    old_v=np.zeros((4,4))
    old_v[-1,-1]=1
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env,policy,old_v,gamma)
        new_policy = extract_policy(old_policy_v,gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [14]:
def show_policy(policy):
    ans=[]
    for p in policy:
        if p==0:
            ans.append('left')
        elif p==1:
            ans.append('down')
        elif p==2:
            ans.append('right')
        else:
            ans.append('up')
    return np.array(ans).reshape(4,4)
            

In [15]:
gamma = 1.0
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env.env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)

print(policy_score)

0.07


In [16]:
show_policy(optimal_policy)

array([['down', 'right', 'down', 'left'],
       ['down', 'down', 'down', 'left'],
       ['right', 'down', 'down', 'down'],
       ['right', 'right', 'right', 'down']], dtype='<U5')

In [17]:
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG
